In [27]:
import os
import random
import pickle
import gc
import warnings
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN
from sklearn.preprocessing import (
    StandardScaler, PowerTransformer, OrdinalEncoder,
    OneHotEncoder, FunctionTransformer, PolynomialFeatures
)
from category_encoders import TargetEncoder, BinaryEncoder
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import (
    LogisticRegression, LinearRegression, Ridge, Lasso,
    SGDRegressor, ElasticNet, Perceptron
)
from sklearn.model_selection import (
    train_test_split, cross_val_score, cross_validate,
    GridSearchCV, KFold, cross_val_predict , RandomizedSearchCV
)
from sklearn.metrics import (
    roc_auc_score, mean_squared_error, make_scorer, accuracy_score
)
from sklearn import set_config, datasets
from catboost import (
    CatBoostRegressor, CatBoostClassifier
)
from sklearn.pipeline import (
    Pipeline, FeatureUnion, make_pipeline
)
from sklearn.ensemble import (
    RandomForestClassifier, StackingClassifier, StackingRegressor,
    GradientBoostingRegressor, VotingClassifier, VotingRegressor,
    HistGradientBoostingRegressor, GradientBoostingClassifier,
    BaggingClassifier, AdaBoostClassifier
)
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.svm import SVC, SVR, LinearSVC
from xgboost import XGBRegressor, XGBClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons, load_iris
import tensorflow as tf
from tensorflow import keras
from scipy.stats import reciprocal

import xgboost as xgb
import lightgbm as lgb
import category_encoders as ce
import re
import optuna
import math

%matplotlib inline

warnings.filterwarnings("ignore")

In [ ]:

X = 2 * np.random.rand(100,1)
y = 4 + 3*X + np.random.randn(100,1)

In [ ]:
X_b = np.c_[np.ones((100,1)),X]
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

In [ ]:
theta_best

In [ ]:
X_new =np.array([[0],[2]])
X_new_b = np.c_[np.ones((2,1)),X_new]
y_predict = X_new_b.dot(theta_best)
y_predict

In [ ]:
plt.plot(X_new,y_predict,"r-")
plt.plot(X,y,"b.")
plt.axis([0,2,0,15])
plt.show

In [ ]:
lin_reg=LinearRegression()
lin_reg.fit(X,y)

In [ ]:
lin_reg.intercept_,lin_reg.coef_

In [ ]:
lin_reg.predict(X_new)

In [ ]:
# 이 함수는 유사역행렬을 구함 => svd로 계산됨
theta_best_svd, residuals, rank , s = np.linalg.lstsq(X_b,y,rcond = 1e-6)
theta_best_svd

In [ ]:
eta = 0.1
n_iterations = 1000
m= 100
theta = np.random.randn(2,1)

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta)-y)
    theta = theta - eta *gradients

theta

In [ ]:
n_epochs = 50
t0, t1 = 5, 50
def learning_schedule(t):
    return t0/(t+t1)

theta = np.random.randn(2,1)

for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index +1]
        yi = y[random_index:random_index+1]
        gradients = 2*xi.T.dot(xi.dot(theta)-yi)
        eta - learning_schedule(epoch * m +i)
        theta = theta - eta * gradients
theta

In [ ]:
sgd_reg = SGDRegressor(max_iter= 1000, tol=1e-3, penalty= None, eta0 = 0.1)
# 최대 1000번 동안, 0.001보다 적게 손실이 줄어들 때까지, 학습률 0.1
sgd_reg.fit(X,y.ravel())

In [ ]:
sgd_reg.intercept_, sgd_reg.coef_

# 다항회귀

In [ ]:
m = 100
X = 6 * np.random.rand(m,1)- 3
y = 0.5 * X **2 + X + 2 +np.random.randn(m,1)


In [ ]:
poly_features = PolynomialFeatures(degree = 2, include_bias= False)
X_poly = poly_features.fit_transform(X)
X[0]
X_poly[0]

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly,y)
lin_reg.intercept_, lin_reg.coef_

In [ ]:
def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2) 
    train_errors, val_errors = [], [] 
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m]) 
        y_train_predict = model.predict(X_train[:m]) 
        y_val_predict = model.predict(X_val) 
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict)) 
        val_errors.append(mean_squared_error(y_val, y_val_predict)) 
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train set") 
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val set")
    plt.legend(loc="upper right", fontsize=14)

plot_learning_curves(lin_reg , X, y) 

In [ ]:
polynomial_regression = Pipeline ([ 
    ("poly_features" , PolynomialFeatures(degree=10, include_bias=False)), 
    ("lin_reg" , LinearRegression()),
]) 

plot_learning_curves(polynomial_regression, X, y) 

In [ ]:
ridge_reg = Ridge(alpha= 1, solver = 'cholesky')
ridge_reg.fit(X,y)
ridge_reg.predict([[1.5]])

In [ ]:
sgd_reg = SGDRegressor(penalty="l2") 
sgd_reg.fit(X, y.ravel()) 
sgd_reg.predict([[1.5]])

In [ ]:
elastic_net = ElasticNet(alpha= 0.1 , l1_ratio = 0.5)
elastic_net.fit(X,y)
elastic_net.predict([[1.5]])


In [ ]:
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming X and y are defined

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

poly_scaler = Pipeline([
    ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
    ("std_scaler", StandardScaler())
])

X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scaled = poly_scaler.transform(X_val)

sgd_reg = SGDRegressor(max_iter=1, tol=1e-3, warm_start=True, penalty=None, learning_rate="constant", eta0=0.0005)

minimum_val_error = float("inf")
best_epoch = None
best_model = None

for epoch in range(1000):
    sgd_reg.fit(X_train_poly_scaled, y_train)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    val_error = mean_squared_error(y_val, y_val_predict)
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = clone(sgd_reg)  # Corrected this line


In [ ]:
# SVM

iris = datasets.load_iris()
X=iris["data"][:,(2,3)]
y=(iris["target"] == 2).astype(np.float64)

svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1,loss = "hinge"))
])

svm_clf.fit(X,y)
svm_clf.predict([[5.5,1.7]])

In [ ]:
# 비선형 SVM

X,y = make_moons(n_samples = 100,noise = 0.15)
poly_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree = 3)),
    ("Scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C = 10, loss = "hinge"))
])

poly_svm_clf.fit(X,y)

In [ ]:
# 가우시안 RBF 커널
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel = "rbf", gamma = 5, C = 0.001))

])
rbf_kernel_svm_clf.fit(X,y)

In [ ]:
# Decision Tree 
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data[:,2:]
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth= 2)
tree_clf.fit(X,y)


In [ ]:
# Voting
X, y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()  
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),n_estimators= 500,
    max_samples= 100, bootstrap= True, n_jobs= -1,oob_score= True)

bag_clf.fit(X_train,y_train)
bag_clf.oob_score_
y_pred = bag_clf.predict(X_test)

In [ ]:
# AdaBoostClassifier
ada_clf = AdaBoostClassifier( 
    DecisionTreeClassifier(max_depth=1), n_estimators=200, 
    algorithm= "SAMME.R", learning_rate=0.5) 
ada_clf.fit (X_train, y_train) 

In [ ]:
# XGBoost
X, y = make_moons(n_samples=100, noise=0.15)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

xgb_reg = XGBRegressor()
xgb_reg.fit(X_train,y_train,eval_set = [(X_val,y_val)], early_stopping_rounds =2)
y_pred = xgb_reg.predict(X_val)

In [ ]:
# PCA 
pca = PCA(n_components= 2)
X2D= pca.fit_transform(X)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# 차원 수 선택
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

In [ ]:
pca = PCA(n_components = d)
X_reduced = pca.fit_transform(X_train)

In [ ]:
rnd_pca = PCA(n_components= 154, svd_solver= 'randomized')
X_reduced = rnd_pca.fit_transform(X_train)

In [ ]:
# # IPAC  데이터가 컬럼수가 2개 밖에 없음
# n_batches = 100
# inc_pca = IncrementalPCA(n_components= 154)
# for X_batch in np.array_split(X_train, n_batches):
#     inc_pca.partial_fit(X_batch)

# X_reduced = inc_pca.tranform(X_train)

In [ ]:
rbf_pca = KernelPCA(n_components= 2,kernel='rbf', gamma = 0.04)
X_reduced = rbf_pca.fit_transform(X_train)

In [ ]:
# 커널 선택과 하이퍼파라미터 튜닝 
clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
])

param_grid = [{
    "kpca__gamma" : np.linspace(0.03,0.05,10),
    "kpca__kernel" : ['rbf','sigmoid']
}]

grid_search = GridSearchCV(clf,param_grid, cv= 3)
grid_search.fit(X,y)

print(grid_search.best_params_)

In [ ]:
# 재구성 원상
rbf_pca = KernelPCA(n_components=2, kernel='rbf', gamma= 0.0433,
                    fit_inverse_transform= True) # 자동으로 수행됨
X_reduced = rbf_pca.fit_transform(X_train)
X_preimage = rbf_pca.inverse_transform(X_reduced)

mean_squared_error(X, X_preimage)

In [ ]:
lle = LocallyLinearEmbedding(n_components=2,n_neighbors=10)
X_reduced = lle.fit_transform(X)

In [ ]:
# K-Means
k = 5
kmeans = KMeans(n_clusters= k)
y_pred = kmeans.fit_predict(X)

# K-means++

km = KMeans(init ="k-means++", n_clusters= k, random_state= 42)
y_pred = kmeans.fit_predict(X)

# centroid reset
good_init = np.array([[-3,3], [-3,2],[-3,1],[-1,2],[0,2]])
kmeans = KMeans(n_clusters= 5, init= good_init, n_init= 5)

kmeans.inertia_

In [ ]:
minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

In [ ]:
# DBSCAN
X, y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples= 5)
dbscan.fit(X)

# Chapter 2

In [ ]:
# Perceptron
iris = load_iris()
X= iris.data[:,(2,3)]
y= (iris.target == 0).astype(int)

per_clf = Perceptron()
per_clf.fit(X,y)

y_pred = per_clf.predict([[2,0.5]])


### Tensorflow

In [ ]:


fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test,y_test) = fashion_mnist.load_data()


In [ ]:
X_train_full.dtype

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0 
y_valid, y_train = y_train_full[:5000], y_train_full[5000:] 
X_test = X_test / 255.0 

In [ ]:
class_names =  ["T-shirt/top", "Trouser", "Pullover", "Dress" , "Coat" , 
"Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot" ]

In [ ]:
class_names[y_train[0]]

In [ ]:
# model = keras.models.Sequential()
# model.add(keras.layers.Flatten(input_shape = [28,28])) # sequential model making, Flatten 입력 이미지를 1D 배열로 변환
# model.add(keras.layers.Dense(300, activation = 'relu')) # 300개의 뉴런 dense 은닉층 추가, 
# model.add(keras.layers.Dense(100, activation = 'relu'))
# model.add(keras.layers.Dense(10, activation = 'softmax')) # 클래스마다 하나씩인 뉴런 10개를 가진 Dense 출력층 추가, 


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28,28]),
    keras.layers.Dense(300, activation = 'relu'),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

In [ ]:
model.summary()

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[1] # 모델에 있는 층의 리스트를 출력하거나 인덱스, 이름으로 층을 선택할 수 있다
hidden1.name

In [ ]:
model.get_layer('dense') is hidden1

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy", 
              optimizer = 'sgd',
              metrics=['accuracy'])
''' 레이블이 정수 하나로 이루어져있고 클래스가 배터적이므로 sparse_categorical_crossentropy 사용
    클래스별 타겟 확률을 가지고 있다면 categorical_crossentropy 사용
    이진 분류를 수행한다면 softmax 대신 sigmoid, binary_crossentropy 사용'''

In [ ]:
history = model.fit(X_train, y_train, epochs = 30,
                    validation_data = (X_valid, y_valid))

In [ ]:
pd.DataFrame(history.history).plot(figsize = (8,5))
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

In [ ]:
# y_pred= model.predict_classes(X_new)
# y_pred

## 시퀀셜 api를 사용하여 회귀용 다층 퍼셉트론 만들기

In [17]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [18]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation= 'relu', input_shape = X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss= 'mean_squared_error', optimizer= 'sgd')
history = model.fit(X_train, y_train, epochs = 20,
                    validation_data = (X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 0.7554 - val_loss: 0.5265
Epoch 2/20
363/363 [==============================] - 0s 807us/step - loss: 0.5120 - val_loss: 6.4186
Epoch 3/20
363/363 [==============================] - 0s 818us/step - loss: 0.8861 - val_loss: 9.5896
Epoch 4/20
363/363 [==============================] - 0s 805us/step - loss: 0.7591 - val_loss: 129.7116
Epoch 5/20
363/363 [==============================] - 0s 820us/step - loss: 0.5798 - val_loss: 0.3911
Epoch 6/20
363/363 [==============================] - 0s 804us/step - loss: 0.4437 - val_loss: 0.3959
Epoch 7/20
363/363 [==============================] - 0s 796us/step - loss: 0.3822 - val_loss: 0.4550
Epoch 8/20
363/363 [==============================] - 0s 799us/step - loss: 0.3778 - val_loss: 0.3809
Epoch 9/20
363/363 [==============================] - 0s 831us/step - loss: 0.3682 - val_loss: 0.3960
Epoch 10/20
363/363 [==============================] - 0s 796us/step - loss: 0.374

In [ ]:
input = keras.layers.Input(shape = X_train.shape[1:]) # input 객체를 만들기, shape과 dtype을 포함하여 모델의 입력을 정의
hidden1 = keras.layers.Dense(30, activation= 'relu')(input) # 30개의 뉴런과 Relu 활성화함 수를 가진 Dense 층을 만듬, 이 층은 만들어지자마자 입력과 함께 함수처럼 호출, 이를 함수형 API라고 부름
hidden2 = keras.layers.Dense(30, activation= 'relu')(hidden1) # 두 번째 은닉층을 만들고 함수처럼 호출
concat = keras.layers.Concatenate()([input,hidden2]) # 두 번재 은닉층의 출력과 입력을 연결, 
output = keras.layers.Dense(1)(concat) # 하나의 뉴런과 활성화 함수가 없는 출력층을 만들고 결과를 사용해 호출
model = keras.models.Model(inputs= [input], outputs= output)


In [20]:
input_A = keras.layers.Input(shape = [5],name = "wide_input")
input_B = keras.layers.Input(shape = [6],name = "deep_input")
hiddenl = keras.layers.Dense(30,activation = "relu")(input_B)
hidden2 = keras.layers.Dense(30,activation = "relu")(hiddenl)
concat = keras.layers.concatenate([input_A,hidden2])
output = keras.layers.Dense(1,name = "output")(concat)
model = keras.Model(inputs = [input_A,input_B],outputs = [output])

In [21]:
model.compile(loss='mse', optimizer=keras.optimizers.SGD(learning_rate=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs = 20,
                    validation_data=((X_valid_A, X_valid_B),y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A,X_new_B))

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 2.2808 - val_loss: 0.9920
Epoch 2/20
363/363 [==============================] - 0s 862us/step - loss: 0.7758 - val_loss: 0.6928
Epoch 3/20
363/363 [==============================] - 0s 863us/step - loss: 0.6464 - val_loss: 0.6208
Epoch 4/20
363/363 [==============================] - 0s 865us/step - loss: 0.5964 - val_loss: 0.5794
Epoch 5/20
363/363 [==============================] - 0s 887us/step - loss: 0.5644 - val_loss: 0.5487
Epoch 6/20
363/363 [==============================] - 0s 865us/step - loss: 0.5384 - val_loss: 0.5212
Epoch 7/20
363/363 [==============================] - 0s 871us/step - loss: 0.5193 - val_loss: 0.5052
Epoch 8/20
363/363 [==============================] - 0s 869us/step - loss: 0.5033 - val_loss: 0.4880
Epoch 9/20
363/363 [==============================] - 0s 876us/step - loss: 0.4889 - val_loss: 0.4815
Epoch 10/20
363/363 [==============================] - 0s 856us/step - loss: 0.4856 

In [ ]:
output = keras.layers.Dense(1, name= 'main_output')(concat)
aux_output = keras.layers.Dense(1, name = 'aux_output')(hidden2)
model = keras.Model(inputs = [input_A, input_B], outputs = [output, aux_output])

In [ ]:
model.compile(loss = ['mse','mse'], loss_weights = [0.9, 0.1], optimizer= 'sgd')

In [ ]:
history = model.fit(
    [X_train_A, X_train_B], [y_train, y_train], epochs = 20,
    validation_data = ([X_valid_A, X_valid_B], [y_valid, y_valid])
)

In [ ]:
total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])

In [ ]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

In [ ]:
# 서브클래싱 API로 동적 모델 만들기
class WideAndDeepModel(keras.Model):
    def __init__(self, units = 30, activation = 'relu',**kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units,activation = activation)
        self.hidden2 = keras.layers.Dense(units,activation = activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    def call(self,inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_out = self.main_output(concat)
        aux_out = self.aux_output(hidden2)
        return main_out, aux_out

model = WideAndDeepModel()

In [ ]:
model.save('my_model.keras')

In [ ]:
model = keras.models.load_model('my_model.keras')

In [ ]:
import winsound
frequency = 200  # 소리의 주파수 (Hz)
duration = 1000  # 소리의 지속 시간 (밀리초)
winsound.Beep(frequency, duration)

# 콜백 사용

In [ ]:
#[...]  모델 만들고 컴파일
checkpoints_cb = keras.callbacks.ModelCheckpoint('my_model.keras')
history = model.fit(X_train, y_train, epochs = 10, callbacks = [checkpoints_cb])

In [ ]:
checkpoints_cb = keras.callbacks.ModelCheckpoint('my_model.keras', save_best_only = True)
history = model.fit(X_train, y_train, epochs = 10,
                    validation_data = (X_valid, y_valid),
                    callbacks = [checkpoints_cb])
model = keras.models.load_model('my_model.keras')

## EarlyStopping

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights= True)
history = model.fit(X_train, y_train, epochs = 100,
                    validation_data = (X_valid, y_valid),
                    callbacks = [checkpoints_cb, early_stopping_cb])


In [ ]:
class PrintValTrainRatioCallback(keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs): 
        print("\nval/train:{:. 2f}" . format ( logs[ "va l_loss "] / logs[" loss "])) 

In [ ]:
import os
root_logdir = os.path.join(os.curdir, 'logs')

def get_run_logdir():
    import time
    run_id = time.strftime('run_%y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation= 'relu', input_shape = X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss= 'mean_squared_error', optimizer= 'sgd')

In [ ]:
# tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
# history = model.fit(X_train, y_train, epochs = 30,
#                     validation_data= (X_valid, y_valid),
#                     callbacks = [tensorboard_cb])

# 신경망 하이퍼파라미터 튜닝

In [14]:
def build_model(n_hidden = 1, n_neurons = 30, learning_rate = 3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape = input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation = 'relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate = learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model


In [15]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
keras_reg= keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [22]:
keras_reg.fit(X_train, y_train, epochs=100, 
        validation_data=(X_valid, y_valid), 
        callbacks=[keras.callbacks.EarlyStopping(patience=10)]) 
mse_test = keras_reg.score(X_test, y_test) 
y_pred = keras_reg.predict(X_new) 

Epoch 1/100
363/363 [==============================] - 1s 1ms/step - loss: 1.3607 - val_loss: 5.0955
Epoch 2/100
363/363 [==============================] - 0s 815us/step - loss: 0.9390 - val_loss: 0.7575
Epoch 3/100
363/363 [==============================] - 0s 818us/step - loss: 0.5727 - val_loss: 0.5492
Epoch 4/100
363/363 [==============================] - 0s 819us/step - loss: 0.5088 - val_loss: 0.4908
Epoch 5/100
363/363 [==============================] - 0s 878us/step - loss: 0.4776 - val_loss: 0.4614
Epoch 6/100
363/363 [==============================] - 0s 870us/step - loss: 0.4590 - val_loss: 0.4456
Epoch 7/100
363/363 [==============================] - 0s 840us/step - loss: 0.4467 - val_loss: 0.4365
Epoch 8/100
363/363 [==============================] - 0s 825us/step - loss: 0.4371 - val_loss: 0.4324
Epoch 9/100
363/363 [==============================] - 0s 809us/step - loss: 0.4308 - val_loss: 0.4266
Epoch 10/100
363/363 [==============================] - 0s 804us/step - los

In [29]:
param_distribs = {
    'n_hidden' : [0,1,2,3],
    'n_neurons' : np.arange(1,100),
    'learning_rate' : reciprocal(3e-4,3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg,param_distribs, n_iter =10, cv= 3)
rnd_search_cv.fit(X_train, y_train, epochs = 100,
                  validation_data= (X_valid, y_valid),
                  callbacks = [keras.callbacks.EarlyStopping(patience = 10)])


Epoch 1/100
242/242 [==============================] - 1s 2ms/step - loss: 5.1429 - val_loss: 3.2212
Epoch 2/100
242/242 [==============================] - 0s 945us/step - loss: 2.0715 - val_loss: 1.5017
Epoch 3/100
242/242 [==============================] - 0s 919us/step - loss: 1.2341 - val_loss: 1.0197
Epoch 4/100
242/242 [==============================] - 0s 894us/step - loss: 0.9330 - val_loss: 0.8429
Epoch 5/100
242/242 [==============================] - 0s 902us/step - loss: 0.8121 - val_loss: 0.7693
Epoch 6/100
242/242 [==============================] - 0s 902us/step - loss: 0.7573 - val_loss: 0.7278
Epoch 7/100
242/242 [==============================] - 0s 902us/step - loss: 0.7276 - val_loss: 0.7029
Epoch 8/100
242/242 [==============================] - 0s 900us/step - loss: 0.7076 - val_loss: 0.6872
Epoch 9/100
242/242 [==============================] - 0s 903us/step - loss: 0.6929 - val_loss: 0.6710
Epoch 10/100
242/242 [==============================] - 0s 914us/step - los

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x00000233DFB32450>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000233E7C0DB90>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [30]:
rnd_search_cv.best_params_

{'learning_rate': 0.005657021615090343, 'n_hidden': 3, 'n_neurons': 44}

In [31]:
rnd_search_cv.best_score_

-0.2985316812992096